# First Name: 
# Last Name: 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi 
import matplotlib.pyplot as plt

In [2]:
nesarc = pd.read_csv('nesarc.csv', low_memory=False)
pd.set_option('display.float_format', lambda x:'%f'%x) 

In [6]:
nesarc

,S3BQ1A1,ETHRACE2A,ETOTLCA2,IDNUM,PSU,STRATUM,WEIGHT,CDAY,CMON,CYEAR,...,SOLP12ABDEP,HAL12ABDEP,HALP12ABDEP,MAR12ABDEP,MARP12ABDEP,HER12ABDEP,HERP12ABDEP,OTHB12ABDEP,OTHBP12ABDEP,NDSymptoms
0,0,5,,1,4007,403,3928.613505,14,8,2001,...,0,0,0,0,0,0,0,0,0,nan
1,1,5,0.0014,2,6045,604,3638.691845,12,1,2002,...,0,0,0,0,0,0,0,0,0,nan
2,2,5,,3,12042,1218,5779.032025,23,11,2001,...,0,0,0,0,0,0,0,0,0,nan
3,3,5,,4,17099,1704,1071.754303,9,9,2001,...,0,0,0,0,0,0,0,0,0,nan
4,4,2,,5,17099,1704,4986.952377,18,10,2001,...,0,0,0,0,0,0,0,0,0,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43088,43088,1,,43089,12010,1208,10477.240840,27,11,2001,...,0,0,0,0,0,0,0,0,0,nan
43089,43089,1,0.2237,43090,17099,1704,9014.746280,30,10,2001,...,0,0,0,0,0,0,0,0,0,nan
43090,43090,1,0.3785,43091,18094,1802,8079.917091,16,10,2001,...,0,0,0,0,0,0,0,0,0,nan
43091,43091,1,14.0831,43092,31035,3104,10367.259020,26,9,2001,...,3,0,3,0,3,0,0,0,0,nan


In [ ]:
nesarc['S3AQ3B1'] = pd.to_numeric(nesarc['S3AQ3B1']) #convert variable to numeric
nesarc['S3AQ3C1'] = pd.to_numeric(nesarc['S3AQ3C1']) #convert variable to numeric
nesarc['CHECK321'] = pd.to_numeric(nesarc['CHECK321']) #convert variable to numeric

In [ ]:
sub1=nesarc[(nesarc['AGE']>=18) & (nesarc['AGE']<=25) & (nesarc['CHECK321']==1)]
sub2=sub1.copy()

In [ ]:
#SETTING MISSING DATA
sub2['S3AQ3B1']=sub2['S3AQ3B1'].replace(9, np.nan)
sub2['S3AQ3C1']=sub2['S3AQ3C1'].replace(99, np.nan)

In [ ]:
recode2 = {1: 30, 2: 22, 3: 14, 4: 5, 5: 2.5, 6: 1}
sub2['USFREQMO']= sub2['S3AQ3B1'].map(recode2)
sub2['USFREQMO']= pd.to_numeric(sub2['USFREQMO'])

In [ ]:
# Creating a secondary variable multiplying the days smoked/month and the number of cig/per day
sub2['NUMCIGMO_EST']=sub2['USFREQMO'] * sub2['S3AQ3C1']
sub2['NUMCIGMO_EST']= pd.to_numeric(sub2['NUMCIGMO_EST'])

In [ ]:
ct1 = sub2.groupby('NUMCIGMO_EST').size()
print (ct1)

# Categorical -> Quantitative - ANOVA

In [ ]:
sub2['MAJORDEPLIFE'] = sub2['MAJORDEPLIFE'].astype('category') 

In [ ]:
%matplotlib notebook
sns.boxplot(x='MAJORDEPLIFE', y='NUMCIGMO_EST', data=sub2)
plt.xlabel('MAJORDEPLIFE')
plt.ylabel('NUMCIGMO_EST')

In [ ]:
model1 = smf.ols(formula='NUMCIGMO_EST ~ C(MAJORDEPLIFE)', data=sub2).fit()
print (model1.summary())

In [ ]:
sub3 = sub2[['NUMCIGMO_EST', 'MAJORDEPLIFE']].dropna()

In [ ]:
print ('means for numcigmo_est by major depression status')
m1= sub3.groupby('MAJORDEPLIFE').mean()
print (m1)

In [ ]:
print ('standard deviations for numcigmo_est by major depression status')
sd1 = sub3.groupby('MAJORDEPLIFE').std()
print (sd1)

# Categorical (>2) -> Quantitative - ANOVA

In [ ]:
sub2['ETHRACE2A'] = sub2['ETHRACE2A'].astype('category') 
sub2['ETHRACE2A']=sub2['ETHRACE2A'].cat.rename_categories(["White", "Black", "NatAm", "Asian", "Hispanic"])

In [ ]:
%matplotlib notebook
sns.boxplot(x='ETHRACE2A', y='NUMCIGMO_EST', data=sub2)
plt.xlabel('ETHRACE2A')
plt.ylabel('NUMCIGMO_EST')

In [ ]:
sub4 = sub2[['NUMCIGMO_EST', 'ETHRACE2A']].dropna()

In [ ]:
model2 = smf.ols(formula='NUMCIGMO_EST ~ C(ETHRACE2A)', data=sub4).fit()
print (model2.summary())

In [ ]:
print ('means for numcigmo_est by ethnicity')
m2= sub4.groupby('ETHRACE2A').mean()
print (m2)

In [ ]:
print ('standard deviations for numcigmo_est by ethnicity')
sd2 = sub4.groupby('ETHRACE2A').std()
print (sd2)

In [ ]:
mc1 = multi.MultiComparison(sub4['NUMCIGMO_EST'], sub4['ETHRACE2A'])
res1 = mc1.tukeyhsd()
print(res1.summary())